# 실행모드(ESC)
* 커서가 사라집니다.
* a키를 누르면 위에 셀이 생성됩니다.
* b키를 누르면 아래에 셀이 생성됩니다.
* dd키를 누르면 셀이 삭제 됩니다.
* m키를 누르면 마크다운 셀로 변경됩니다.


# 편집모드(Enter)
* 커서가 생깁니다.

In [ ]:
%pwd

'/home/sjt/projects/web/likelion/AIS8'

In [39]:
import os

os.getcwd()

'/home/sjt/projects/web/likelion/AIS8'

In [40]:
# time : 간격을 두고 가져오기 위해 사용
# pandas : 파이썬에서 사용할 수 있는 엑셀과 유사한 데이터분석 도구
# numpy : 파이썬에서 사용할 수 있는 공학용 계산기
# requests : 매우 작은 브라우저로 웹사이트의 내용과 정보를 불러옴
# BeautifulSoup : requests로 가져온 웹사이트의 html 태그를 찾기위해 사용
import time
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

In [41]:
def get_one_page(page_no):
    base_url = f'https://opengov.seoul.go.kr/civilappeal/list?items_per_page=50&page={page_no}'
    try:
        response = requests.get(base_url)
    except requests.exceptions.HTTPError as errh:
        time.sleep(3)
        try:
            response = requests.get(base_url)
        except requests.exceptions.HTTPError as errh:
            print ("Http Error : ",errh)
    df = pd.read_html(response.text)[0]
    html = bs(response.text)
    text = [i['href'].split('/')[-1] for i in html.select('td.data-title.aLeft > a')]
    try:
        df['내용번호'] = text
    except ValueError as errv:
        print(f'df 길이 : {len(df)} / 리스트 길이 : {len(text)}')
        print('ValueError : ',errv)
    return df

In [42]:
page_no = 1
table_list = []

while True:
    df_one = get_one_page(page_no)
    table_list.append(df_one)
    print(page_no, end=",")
    if len(df_one) < 2 :
        break
    page_no += 1
    time.sleep(0.01)

df = pd.concat(table_list)
df

1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,

In [ ]:
df = df.sort_values(by=['번호'], axis=0).reset_index(drop=True)
df.to_csv('./data/seoul-120/seoul-120-list.csv',index=False)

In [ ]:
# 인증키 : '6d49554463736a7436306357675561'

In [23]:
from urllib.error import HTTPError
import time
import requests
from tqdm import tqdm
tqdm.pandas()
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as bs

def scraping_error_handle(func):
    try:
        return func
    except HTTPError as errh:
        try:
            time.sleep(2)
            return func
        except:
            print(f'http error : {errh}')
    except ValueError as errv:
        print(f'ValueError : {errv}')

def get_desc(response):
    df_desc = pd.read_html(response.text)[1]
    tb01 = df_desc[[0,1]].set_index(0).T.reset_index(drop=True)
    tb02 = df_desc[[2,3]].set_index(2).T.reset_index(drop=True)
    df_tb = pd.concat([tb01,tb02],axis=1,ignore_index=False)
    return df_tb

@scraping_error_handle
def get_view_page(view_no):
    base_url = f'https://opengov.seoul.go.kr/civilappeal/{view_no}'
    response = requests.get(base_url)
    desc = get_desc(response)
    html = bs(response.text)
    text = html.find('div', {'class' : 'line-all'}).text.strip()
    desc['내용'] = text
    desc['내용번호'] = view_no
    time.sleep(0.01)
    return desc

In [24]:
df = pd.read_csv('./data/seoul-120/seoul-120-list.csv')
df = df[-10:].reset_index(drop=True)
df

,번호,제목,생산일,조회수,내용번호
0,2234,서울시 [우리동네꼼꼼육아정보] 책자 발간,2019-10-23,2275,18955676
1,2235,"[성동구] 반려동물 ""무료상담실"" 운영 안내",2020-08-31,1654,21095911
2,2236,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1924,21212235
3,2237,마포 뇌병변장애인 비전센터,2021-03-12,1625,22477798
4,2238,[서울산업진흥원] 서울메이드란?,2021-06-29,8168,23194045
5,2239,다자녀가정 실내 바닥매트 지원,2021-08-17,1319,25670204
6,2240,하반기 농부의 시장 일정은 어떻게 되나요?,2022-08-29,647,26695536
7,2241,핑크퐁 서울공원 대탐험 행사에 참여하려면 예약이 필요한가요?,2022-10-05,48,27097033
8,2242,핑크퐁 서울공원 대탐험 행사는 비가 와도 운영하나요?,2022-10-05,129,27097034
9,2243,수도요금 전자계산서 발급 요청시,2022-11-28,121,27328170


In [25]:
view_no = df['내용번호'][0]
get_view_page(view_no)

,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류,내용,내용번호
0,다산콜센터,120다산콜재단,D0000038442669,서울특별시 여성가족정책실 아이돌봄담당관,2019-10-23,복지,서울시와 서울시육아종합지원센터는 140여 명의 우리동네 보육반장이 직접 발로 뛰어 ...,18955676


In [31]:
view_detail = df['내용번호'].progress_apply(get_view_page)
view_detail = pd.concat(view_detail.to_list()).sort_values('내용번호').reset_index(drop=True)
view_detail

100%|██████████| 10/10 [00:04<00:00,  2.24it/s]


,원본시스템,작성자(책임자),관리번호,제공부서,생산일,분류,내용,내용번호
0,다산콜센터,120다산콜재단,D0000038442669,서울특별시 여성가족정책실 아이돌봄담당관,2019-10-23,복지,서울시와 서울시육아종합지원센터는 140여 명의 우리동네 보육반장이 직접 발로 뛰어 ...,18955676
1,다산콜센터,120다산콜재단,D0000040706778,서울특별시 자치구,2020-08-31,경제,"▣ 반려동물 무료상담실_""무엇이든 물어보시개?"" 운영 안내 구분 내용 ...",21095911
2,다산콜센터,120다산콜재단,D0000040829456,서울특별시 여성가족정책실 여성권익담당관,2020-09-16,복지,업무개요 직장 내 성희롱 지원체계 사각지대에 있는 서울 시민이 안전하고 성평등하...,21212235
3,다산콜센터,120다산콜재단,D0000042114742,서울특별시 복지정책실 복지기획관 장애인복지정책과,2021-03-12,복지,▣ 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학령기 이후 갈...,22477798
4,다산콜센터,120다산콜재단,D0000042894548,서울산업진흥원,2021-06-29,경제,서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일 플랫...,23194045
5,다산콜센터,120다산콜재단,D0000043274644,서울특별시 서초구 주민생활국 여성보육과,2021-08-17,복지,업무개요 다자녀가정 아이들의 안전한 실내 활동과 층간소음 예방을 위해 실내 바닥...,25670204
6,다산콜센터,120다산콜재단,D0000046114132,대외협력과,2022-08-29,경제,○ 하반기 운영일정 구 분 일 정 9월 10월 11월 ...,26695536
7,다산콜센터,120다산콜재단,D0000046546536,서울특별시 푸른도시여가국 공원여가사업과,2022-10-05,행정,"별도 예약이나 신청 절차 없이, 공원에 방문한 누구나 선착순으로 참여가 가능합니다....",27097033
8,다산콜센터,120다산콜재단,D0000046546537,서울특별시 푸른도시여가국 공원여가사업과,2022-10-05,행정,"야외에서 진행되는 행사여서, 비가 많이 오거나 심하게 바람이 부는 등 기상상황에 따...",27097034
9,다산콜센터,120다산콜재단,D0000046828355,요금제도과,2022-11-28,환경,민원인이 세무서 제출용(비용처리 등의 용도) 등으로 수도요금 관련 증빙자료 요청시 ...,27328170


In [32]:
df

,번호,제목,생산일,조회수,내용번호
0,2234,서울시 [우리동네꼼꼼육아정보] 책자 발간,2019-10-23,2275,18955676
1,2235,"[성동구] 반려동물 ""무료상담실"" 운영 안내",2020-08-31,1654,21095911
2,2236,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1924,21212235
3,2237,마포 뇌병변장애인 비전센터,2021-03-12,1625,22477798
4,2238,[서울산업진흥원] 서울메이드란?,2021-06-29,8168,23194045
5,2239,다자녀가정 실내 바닥매트 지원,2021-08-17,1319,25670204
6,2240,하반기 농부의 시장 일정은 어떻게 되나요?,2022-08-29,647,26695536
7,2241,핑크퐁 서울공원 대탐험 행사에 참여하려면 예약이 필요한가요?,2022-10-05,48,27097033
8,2242,핑크퐁 서울공원 대탐험 행사는 비가 와도 운영하나요?,2022-10-05,129,27097034
9,2243,수도요금 전자계산서 발급 요청시,2022-11-28,121,27328170


In [35]:
df_detail = df.merge(view_detail, on=['내용번호','생산일'])
df_detail.columns

Index(['번호', '제목', '생산일', '조회수', '내용번호', '원본시스템', '작성자(책임자)', '관리번호', '제공부서',
       '분류', '내용'],
      dtype='object')

In [36]:
df_detail

,번호,제목,생산일,조회수,내용번호,원본시스템,작성자(책임자),관리번호,제공부서,분류,내용
0,2234,서울시 [우리동네꼼꼼육아정보] 책자 발간,2019-10-23,2275,18955676,다산콜센터,120다산콜재단,D0000038442669,서울특별시 여성가족정책실 아이돌봄담당관,복지,서울시와 서울시육아종합지원센터는 140여 명의 우리동네 보육반장이 직접 발로 뛰어 ...
1,2235,"[성동구] 반려동물 ""무료상담실"" 운영 안내",2020-08-31,1654,21095911,다산콜센터,120다산콜재단,D0000040706778,서울특별시 자치구,경제,"▣ 반려동물 무료상담실_""무엇이든 물어보시개?"" 운영 안내 구분 내용 ..."
2,2236,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1924,21212235,다산콜센터,120다산콜재단,D0000040829456,서울특별시 여성가족정책실 여성권익담당관,복지,업무개요 직장 내 성희롱 지원체계 사각지대에 있는 서울 시민이 안전하고 성평등하...
3,2237,마포 뇌병변장애인 비전센터,2021-03-12,1625,22477798,다산콜센터,120다산콜재단,D0000042114742,서울특별시 복지정책실 복지기획관 장애인복지정책과,복지,▣ 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학령기 이후 갈...
4,2238,[서울산업진흥원] 서울메이드란?,2021-06-29,8168,23194045,다산콜센터,120다산콜재단,D0000042894548,서울산업진흥원,경제,서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일 플랫...
5,2239,다자녀가정 실내 바닥매트 지원,2021-08-17,1319,25670204,다산콜센터,120다산콜재단,D0000043274644,서울특별시 서초구 주민생활국 여성보육과,복지,업무개요 다자녀가정 아이들의 안전한 실내 활동과 층간소음 예방을 위해 실내 바닥...
6,2240,하반기 농부의 시장 일정은 어떻게 되나요?,2022-08-29,647,26695536,다산콜센터,120다산콜재단,D0000046114132,대외협력과,경제,○ 하반기 운영일정 구 분 일 정 9월 10월 11월 ...
7,2241,핑크퐁 서울공원 대탐험 행사에 참여하려면 예약이 필요한가요?,2022-10-05,48,27097033,다산콜센터,120다산콜재단,D0000046546536,서울특별시 푸른도시여가국 공원여가사업과,행정,"별도 예약이나 신청 절차 없이, 공원에 방문한 누구나 선착순으로 참여가 가능합니다...."
8,2242,핑크퐁 서울공원 대탐험 행사는 비가 와도 운영하나요?,2022-10-05,129,27097034,다산콜센터,120다산콜재단,D0000046546537,서울특별시 푸른도시여가국 공원여가사업과,행정,"야외에서 진행되는 행사여서, 비가 많이 오거나 심하게 바람이 부는 등 기상상황에 따..."
9,2243,수도요금 전자계산서 발급 요청시,2022-11-28,121,27328170,다산콜센터,120다산콜재단,D0000046828355,요금제도과,환경,민원인이 세무서 제출용(비용처리 등의 용도) 등으로 수도요금 관련 증빙자료 요청시 ...


In [44]:
file_name = './data/seoul-120/seoul-120-sample.csv'
df_detail.to_csv(file_name,index=False)

In [45]:
df_detail = pd.read_csv(file_name)
df_detail

,번호,제목,생산일,조회수,내용번호,원본시스템,작성자(책임자),관리번호,제공부서,분류,내용
0,2234,서울시 [우리동네꼼꼼육아정보] 책자 발간,2019-10-23,2275,18955676,다산콜센터,120다산콜재단,D0000038442669,서울특별시 여성가족정책실 아이돌봄담당관,복지,서울시와 서울시육아종합지원센터는 140여 명의 우리동네 보육반장이 직접 발로 뛰어 ...
1,2235,"[성동구] 반려동물 ""무료상담실"" 운영 안내",2020-08-31,1654,21095911,다산콜센터,120다산콜재단,D0000040706778,서울특별시 자치구,경제,"▣ 반려동물 무료상담실_""무엇이든 물어보시개?"" 운영 안내 구분 내용 ..."
2,2236,위드유 서울 직장 성희롱.성폭력 예방센터,2020-09-16,1924,21212235,다산콜센터,120다산콜재단,D0000040829456,서울특별시 여성가족정책실 여성권익담당관,복지,업무개요 직장 내 성희롱 지원체계 사각지대에 있는 서울 시민이 안전하고 성평등하...
3,2237,마포 뇌병변장애인 비전센터,2021-03-12,1625,22477798,다산콜센터,120다산콜재단,D0000042114742,서울특별시 복지정책실 복지기획관 장애인복지정책과,복지,▣ 마포뇌병변장애인 비전센터 운영 구분 내용 목적 학령기 이후 갈...
4,2238,[서울산업진흥원] 서울메이드란?,2021-06-29,8168,23194045,다산콜센터,120다산콜재단,D0000042894548,서울산업진흥원,경제,서울의 감성을 담은 다양하고 새로운 경험을 제공하기 위해 만들어진 라이프스타일 플랫...
5,2239,다자녀가정 실내 바닥매트 지원,2021-08-17,1319,25670204,다산콜센터,120다산콜재단,D0000043274644,서울특별시 서초구 주민생활국 여성보육과,복지,업무개요 다자녀가정 아이들의 안전한 실내 활동과 층간소음 예방을 위해 실내 바닥...
6,2240,하반기 농부의 시장 일정은 어떻게 되나요?,2022-08-29,647,26695536,다산콜센터,120다산콜재단,D0000046114132,대외협력과,경제,○ 하반기 운영일정 구 분 일 정 9월 10월 11월 ...
7,2241,핑크퐁 서울공원 대탐험 행사에 참여하려면 예약이 필요한가요?,2022-10-05,48,27097033,다산콜센터,120다산콜재단,D0000046546536,서울특별시 푸른도시여가국 공원여가사업과,행정,"별도 예약이나 신청 절차 없이, 공원에 방문한 누구나 선착순으로 참여가 가능합니다...."
8,2242,핑크퐁 서울공원 대탐험 행사는 비가 와도 운영하나요?,2022-10-05,129,27097034,다산콜센터,120다산콜재단,D0000046546537,서울특별시 푸른도시여가국 공원여가사업과,행정,"야외에서 진행되는 행사여서, 비가 많이 오거나 심하게 바람이 부는 등 기상상황에 따..."
9,2243,수도요금 전자계산서 발급 요청시,2022-11-28,121,27328170,다산콜센터,120다산콜재단,D0000046828355,요금제도과,환경,민원인이 세무서 제출용(비용처리 등의 용도) 등으로 수도요금 관련 증빙자료 요청시 ...
